# Imports and Paths

In [309]:
import urllib
from bs4 import BeautifulSoup, Comment
import os
import pandas as pd
from plotnine import *
import plotnine
import time
import json

plotnine.options.figure_size = (12, 8)
from pandas.api.types import is_string_dtype
pd.options.display.max_columns=150
pd.options.display.max_rows=100

In [3]:
PATH = 'E:/MyDocuments/GitHub/fantasy_football/'

In [3]:
PATH = '/data/msnow/football/'

# Data Dictionary

In [248]:
for cell in rows[0].find_all('th'):
    if cell.has_attr('data-tip'):
        data_dictionary[cell['data-stat']] = cell['data-tip']

In [250]:
with open(f'{PATH}data_dictionary.json', 'w') as f:
    json.dump(data_dictionary, f)

In [249]:
data_dictionary

{'ranker': '<b>Rank</b><br>This is a count of the rows from top to bottom.<br>It is recalculated following the sorting of a column.',
 'age': "Player's age on December 31st of that year",
 'pos': '<b>Position</b><br>In player and team season stats,<br>Capitals indicates primary starter.<br>Lower-case means part-time starter.',
 'g': 'Games played',
 'gs': 'Games started as an offensive or defensive player<br />Numbers are complete for 1920-59<br />and 1970-present but are incomplete otherwise',
 'qb_rec': 'Team record in games started by this QB (regular season)',
 'pass_cmp': 'Passes completed',
 'pass_att': 'Passes attempted',
 'pass_cmp_perc': 'Percentage of Passes Completed<br>Minimum 14 attempts per scheduled game to qualify as leader.<br />Minimum 1500 pass attempts to qualify as career leader.',
 'pass_yds': 'Yards Gained by Passing<br>For teams, sack yardage is deducted from this total',
 'pass_td': 'Passing Touchdowns',
 'pass_td_perc': 'Percentage of Touchdowns Thrown when At

# Teams

## Getting Team pages

In [20]:
base_pg = 'https://www.pro-football-reference.com'
year_pg_tmplt = 'https://www.pro-football-reference.com/years/'
# year_pg_tmplt = 'https://www.pro-football-reference.com/years/1966/'

In [245]:
year = 1966
year_pg_url = os.path.join(year_pg_tmplt,str(year))
year_pg = urllib.request.urlopen(year_pg_url)
soup_year = BeautifulSoup(year_pg, 'html.parser')

In [260]:
teams = soup_year.find("table", {'id':'NFL'}).find_all('a')
team_number = 0

## Extracting info

In [351]:
df_dict.keys()

dict_keys(['df_1966_00', 'df_1966_01', 'df_1966_02', 'df_1966_03', 'df_1966_04', 'df_1966_05', 'df_1966_06', 'df_1966_07', 'df_1966_08', 'df_1966_09', 'df_1966_10', 'df_1966_11', 'df_1966_12', 'df_1966_13', 'df_1966_14', 'df_1967_00', 'df_1967_01', 'df_1967_02', 'df_1967_03', 'df_1967_04', 'df_1967_05', 'df_1967_06', 'df_1967_07', 'df_1967_08', 'df_1967_09', 'df_1967_10', 'df_1967_11', 'df_1967_12', 'df_1967_13', 'df_1967_14', 'df_1967_15', 'df_1968_00', 'df_1968_01', 'df_1968_02', 'df_1968_03', 'df_1968_04', 'df_1968_05', 'df_1968_06', 'df_1968_07', 'df_1968_08', 'df_1968_09', 'df_1968_10', 'df_1968_11', 'df_1968_12', 'df_1968_13', 'df_1968_14', 'df_1968_15', 'df_1969_00', 'df_1969_01', 'df_1969_02', 'df_1969_03', 'df_1969_04', 'df_1969_05', 'df_1969_06', 'df_1969_07', 'df_1969_08', 'df_1969_09', 'df_1969_10', 'df_1969_11', 'df_1969_12', 'df_1969_13', 'df_1969_14', 'df_1969_15', 'df_1970_00', 'df_1970_01', 'df_1970_02', 'df_1970_03', 'df_1970_04', 'df_1970_05', 'df_1970_06', 'df_1970_

In [352]:
# df_dict2 = df_dict.copy()
df_teams = pd.concat(df_dict.values(), ignore_index=True)
df_teams.to_feather(f'{PATH}df_teams_1966_2017.feather')

In [330]:
df_some.shape

(9002, 26)

In [344]:
year, x

(1992, 0)

In [350]:
# df_dict = {}
for year in range(1992,2018):
    year_pg_url = os.path.join(year_pg_tmplt,str(year))
    year_pg = urllib.request.urlopen(year_pg_url)
    soup_year = BeautifulSoup(year_pg, 'html.parser')
    if year < 1970:
        teams = soup_year.find("table", {'id':'NFL'}).find_all('a')
    else:
        teams = soup_year.find("table", {'id':'AFC'}).find_all('a') + soup_year.find("table", {'id':'NFC'}).find_all('a')
    for x in range(len(teams)):
        df_dict['df_'+ str(year) + '_' + str(x).zfill(2)] = extract_from_year(base_pg, soup_year, teams, x) 
        time.sleep(1)
df_all = pd.concat(df_dict.values(), ignore_index=True)

In [263]:
def extract_from_year(base_pg, soup_year, teams, team_number):
    team_pg_url = base_pg + teams[team_number].get('href')
    team_pg = urllib.request.urlopen(team_pg_url)
    soup_team = BeautifulSoup(team_pg, 'lxml')
    game_data = extract_from_team(soup_team)
    df = pd.DataFrame.from_dict(game_data)
    return df

In [323]:
def extract_from_team(soup_team):
    game_data = {}
    comments=soup_team.find_all(string=lambda text:isinstance(text,Comment))
    game_data, game_sz = results(game_data, soup_team)
    game_data = kicks(game_data,comments, game_sz)
    game_data = fumbl(game_data,comments, game_sz)
    game_data = other_td(game_data,comments, game_sz)
    game_data = blocked_punts(game_data,comments, game_sz)
    return game_data

### Game results

In [60]:
table = soup_team.find('table', {'id':'games'})
table_rows = table.find_all('tr')

In [272]:
base_info

['\n',
 <span>1966</span>,
 '\n',
 <span>Dallas Cowboys</span>,
 '\n',
 <span class="header_end">Statistics &amp; Players</span>,
 '\n']

In [349]:
def results(game_data, soup_team):
    table = soup_team.find('table', {'id':'games'})
    table_rows = table.find_all('tr')
    game_date = []
    skip_idx = -999
    for idx, val in enumerate(table.find_all('td',{'data-stat':'game_date'})):
        if val.string=='Playoffs':
            break
        try:
            game_date.append(val['csk'])
        except:
            skip_idx = idx
            pass
    game_data['game_date'] = game_date
    game_sz = len(game_date)
    if skip_idx > 0:
        game_sz_games = game_sz + 1
    else:
        game_sz_games = game_sz


    var_list = ['opp','pts_off','pts_def','first_down_off','yards_off','pass_yds_off','rush_yds_off','to_off','first_down_def','yards_def','pass_yds_def','rush_yds_def','to_def']
    num_set = {'pts_off','pts_def','first_down_off','yards_off','pass_yds_off','rush_yds_off','to_off','first_down_def','yards_def','pass_yds_def','rush_yds_def','to_def'}
    for var_name in var_list:
        var_val_list = []
        for idx,val in enumerate(table.find_all('td',{'data-stat':var_name})[:game_sz_games]):
            if idx != skip_idx:
                if val.string is None:
                    var_val_list.append(0)
                elif var_name in num_set:
                    var_val_list.append(int(val.string))
                else:
                    var_val_list.append(val.string)
        game_data[var_name] = var_val_list
    
    team_name = soup_team.find('h1', {'itemprop':'name'})
    base_info = [x for x in team_name.children]
    game_data['team'] = [base_info[3].string]*game_sz
    game_data['Year'] = [int(base_info[1].string)]*game_sz
    return game_data, game_sz

### Kicks and punts

In [171]:
comments=soup_team.find_all(string=lambda text:isinstance(text,Comment))

In [321]:
def kicks(game_data,comments,game_sz):       
    for v in comments:
        if v.find('div_returns') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    for x in table.find('td',text='Team Total').next_siblings:
        if x['data-stat'] == 'punt_ret_td':
            if x.string is None:
                game_data['punt_ret_td'] = [0]*game_sz
            else:
                game_data['punt_ret_td'] = [int(x.string)/game_sz]*game_sz
        if x['data-stat'] == 'kick_ret_td':
            if x.string is None:
                game_data['kick_ret_td'] = [0]*game_sz
            else:
                game_data['kick_ret_td'] = [int(x.string)/game_sz]*game_sz
    return game_data

### Fumbles and sacks

In [320]:
def fumbl(game_data,comments,game_sz):   
    for v in comments:
        if v.find('div_defense') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    tst_dict = {}
    fumbl_set = {'sacks','fumbles_rec_td','safety_md','fumbles_rec','def_int'}
    for x in table.find('td',text='Team Total').next_siblings:
        if x['data-stat'] in fumbl_set:
            if x.string is None:
                game_data[x['data-stat']] = [0]*game_sz
            else:
                game_data[x['data-stat']] = [int(x.string)/game_sz]*game_sz
    return game_data

### Other TDs

In [319]:
def other_td(game_data,comments,game_sz):
    for v in comments:
        if v.find('div_scoring') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    tst_dict = {}
    other_set = {'ditd','otd'}
    other_dict = {'ditd':'int_td','otd':'blocked_fg_td'}
    for x in table.find('td',text='Team Total').next_siblings:
        if x['data-stat'] in other_set:
            if x.string is None:
                game_data[other_dict[x['data-stat']]] = [0]*game_sz
            else:
                game_data[other_dict[x['data-stat']]] = [int(x.string)/game_sz]*game_sz
    return game_data

### Blocked punts

In [318]:
def blocked_punts(game_data,comments,game_sz):
    for v in comments:
        if v.find('div_kicking') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    tst_dict = {}
    blocked_set = {'punt_blocked'}
    for x in table.find('td',text='Opp Total').next_siblings:
        if x['data-stat'] in blocked_set:
            if x.string is None:
                game_data[x['data-stat']] = [0]*game_sz
            else:
                game_data[x['data-stat']] = [int(x.string)/game_sz]*game_sz
    return game_data

## DataFrame

In [219]:
df = pd.DataFrame.from_dict(game_data)
df

,Year,blocked_fg_td,def_int,first_down_def,first_down_off,fumbles_rec,fumbles_rec_td,game_date,int_td,kick_ret_td,...,punt_ret_td,rush_yds_def,rush_yds_off,sacks,safety_md,team,to_def,to_off,yards_def,yards_off
0,1966,0,1.214286,18,24,1.928571,0.071429,1966-09-18,0.214286,0.071429,...,0.0,137,144,4.285714,0.071429,Dallas Cowboys,4,1,257,518
1,1966,0,1.214286,25,18,1.928571,0.071429,1966-09-25,0.214286,0.071429,...,0.0,142,148,4.285714,0.071429,Dallas Cowboys,1,0,389,314
2,1966,0,1.214286,20,22,1.928571,0.071429,1966-10-02,0.214286,0.071429,...,0.0,106,143,4.285714,0.071429,Dallas Cowboys,5,2,276,363
3,1966,0,1.214286,11,32,1.928571,0.071429,1966-10-09,0.214286,0.071429,...,0.0,38,212,4.285714,0.071429,Dallas Cowboys,1,0,217,652
4,1966,0,1.214286,13,17,1.928571,0.071429,1966-10-16,0.214286,0.071429,...,0.0,45,135,4.285714,0.071429,Dallas Cowboys,0,4,175,326
5,1966,0,1.214286,17,30,1.928571,0.071429,1966-10-23,0.214286,0.071429,...,0.0,105,156,4.285714,0.071429,Dallas Cowboys,0,4,264,413
6,1966,0,1.214286,6,24,1.928571,0.071429,1966-10-30,0.214286,0.071429,...,0.0,7,192,4.285714,0.071429,Dallas Cowboys,4,3,119,425
7,1966,0,1.214286,5,16,1.928571,0.071429,1966-11-06,0.214286,0.071429,...,0.0,58,112,4.285714,0.071429,Dallas Cowboys,3,2,80,240
8,1966,0,1.214286,19,24,1.928571,0.071429,1966-11-13,0.214286,0.071429,...,0.0,23,132,4.285714,0.071429,Dallas Cowboys,0,3,341,515
9,1966,0,1.214286,16,16,1.928571,0.071429,1966-11-20,0.214286,0.071429,...,0.0,53,124,4.285714,0.071429,Dallas Cowboys,3,1,163,258


# Players

## Pages

In [4]:
bs_pg = 'https://www.pro-football-reference.com/years'
# scoring = {'pg_end':'scoring.htm', 'id':'scoring'}
# returns = {'pg_end':'returns.htm', 'id':'scoring'}
# kicking = {'pg_end':'kicking.htm', 'id':'scoring'}
# defense = {'pg_end':'defense.htm', 'id':'scoring'}
# rushing = {'pg_end':'rushing.htm', 'id':'scoring'}
# passing = {'pg_end':'passing.htm', 'id':'scoring'}
# receiving = {'pg_end':'receiving.htm', 'id':'scoring'}
bs_scoring = 'scoring.htm'
bs_returns = 'returns.htm'
bs_kicking = 'kicking.htm'
bs_defense = 'defense.htm'
bs_rushing = 'rushing.htm'
bs_passing = 'passing.htm'
bs_receiving = 'receiving.htm'

In [136]:
year = 2017
pg_url = os.path.join(bs_pg,str(year),bs_defense)
pg = urllib.request.urlopen(pg_url)
soup = BeautifulSoup(pg, 'html.parser')

In [34]:
rename_dict = {'g':'games','gs':'games_started','prtd':'punt_td','krtd':'ko_td','frtd':'fum_td','otd':'block_rtd',
               'ditd':'int_td','rushtd':'rush_td','rectd':'rec_td','alltd':'all_td','gwd':'game_winning_drives'}

### Data Dictionary

In [1]:
data_dictionary = {}
categ_dict = {}
bs_scoring = 'scoring.htm'
bs_returns = 'returns.htm'
bs_kicking = 'kicking.htm'
bs_defense = 'defense.htm'
bs_rushing = 'rushing.htm'
bs_passing = 'passing.htm'
bs_receiving = 'receiving.htm'

In [134]:
year = 2017
pg_url = os.path.join(bs_pg,str(year),bs_receiving)
pg = urllib.request.urlopen(pg_url)
soup = BeautifulSoup(pg, 'html.parser')
table = soup.find('table')
rows = table.find_all('tr')

In [135]:
rows[0].find_all('th')

[<th aria-label="RankThis is a count of the rows from top to bottom.It is recalculated following the sorting of a column." class="ranker poptip sort_default_asc show_partial_when_sorting center" data-stat="ranker" data-tip="&lt;b&gt;Rank&lt;/b&gt;&lt;br&gt;This is a count of the rows from top to bottom.&lt;br&gt;It is recalculated following the sorting of a column." scope="col">Rk</th>,
 <th aria-label="player" class=" poptip sort_default_asc show_partial_when_sorting left" data-stat="player" scope="col"></th>,
 <th aria-label="Tm" class=" poptip sort_default_asc show_partial_when_sorting left" data-stat="team" scope="col">Tm</th>,
 <th aria-label="Player's age on December 31st of that year" class=" poptip sort_default_asc center" data-stat="age" data-tip="Player's age on December 31st of that year" scope="col">Age</th>,
 <th aria-label="PositionIn player and team season stats,Capitals indicates primary starter.Lower-case means part-time starter." class=" poptip sort_default_asc left" 

In [49]:
categ_dict['receiving'] = [x['data-stat'] for x in rows[0].find_all('th')]
for cell in rows[0].find_all('th'):
    if cell.has_attr('data-tip'):
        data_dictionary[cell['data-stat']] = cell['data-tip']
    else:
        data_dictionary[cell['data-stat']] = cell['data-stat']

In [50]:
data_dictionary

{'ranker': '<b>Rank</b><br>This is a count of the rows from top to bottom.<br>It is recalculated following the sorting of a column.',
 'age': "Player's age on December 31st of that year",
 'pos': '<b>Position</b><br>In player and team season stats,<br>Capitals indicates primary starter.<br>Lower-case means part-time starter.',
 'g': 'Games played',
 'gs': 'Games started as an offensive or defensive player<br />Numbers are complete for 1920-59<br />and 1970-present but are incomplete otherwise',
 'otd': 'Other touchdowns from blocked kicks or missed field goals returned',
 'alltd': 'All touchdowns scored',
 'two_pt_md': 'Two-Point Conversions Made',
 'two_pt_att': 'Two-Point Conversions attempted',
 'xpm': 'Extra Points Made',
 'xpa': 'Extra Points Attempted',
 'fgm': 'Total Field Goals Made',
 'fga': 'Field Goals Attempted',
 'safety_md': 'Safeties scored by player/team',
 'scoring': 'Total points scored by all means',
 'points_per_g': 'Points per game',
 'player': 'player',
 'team': '

In [18]:
data_dictionary['ditd'] = 'Interception resulting in a touchdown'

In [137]:
with open(f'{PATH}data_dictionary.json', 'w') as f:
    json.dump(data_dictionary, f)

with open(f'{PATH}categ_dictionary.json', 'w') as f:
    json.dump(categ_dict, f)    

### Scoring

In [8]:
for yr in range(1970,1939,-1):
    pg_url = os.path.join(bs_pg,str(yr),bs_scoring)
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    scoring_list += plyr_scr(soup)
    time.sleep(1)

In [113]:
df_scr = pd.DataFrame.from_dict(scoring_list)
df_scr.rename(columns=rename_dict, inplace=True)
df_scr.to_feather(f'{PATH}df_players_scrape_scoring.feather')
df_scr.shape, df_scr.player_id.nunique(), df_scr.player.nunique()

((23813, 28), 7648, 7493)

In [102]:
def plyr_scr(soup):
    year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
    table = soup.find('table', {'id':'scoring'})
    rows = table.find_all('tr')
    scoring_list = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            tmp_dict = {}
            for cell in cells:
                if cell['data-stat'] == 'player':
                    name = cell.find('a').string
                    tmp_dict[cell['data-stat']] = name
                    tmp_dict['first_name'] = name.split()[0]
                    tmp_dict['last_name'] = name.split()[1]
                    tmp_dict['player_id'] = cell['data-append-csv'].lower()
                elif cell['data-stat'] == 'team':
                    try:
                        tmp_dict[cell['data-stat']] = cell.find('a')['title']
                        tmp_dict['team_id'] = cell.find('a').string.lower()
                    except:
                        tmp_dict[cell['data-stat']] = 'other'
                        tmp_dict['team_id'] = 'other'
                elif cell['data-stat'] == 'pos':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 'no_pos'
                    else:
                        tmp_dict[cell['data-stat']] = val
                else:
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    elif val.isnumeric():
                        tmp_dict[cell['data-stat']] = int(val)
                    else:
                        try:
                            tmp_dict[cell['data-stat']] = float(val)
                        except:
                            tmp_dict[cell['data-stat']] = val
                tmp_dict['year'] = year
            scoring_list.append(tmp_dict)
    return scoring_list

### Passing

In [111]:
for yr in range(2010,1968,-1):
    pg_url = os.path.join(bs_pg,str(yr),bs_passing)
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    passing_list += plyr_pass(soup)
    time.sleep(1)

In [112]:
df_pass = pd.DataFrame.from_dict(passing_list)
df_pass.rename(columns=rename_dict, inplace=True)
df_pass.to_feather(f'{PATH}df_players_scrape_passing.feather')
df_pass.shape, df_pass.player_id.nunique(), df_pass.player.nunique()

((4907, 33), 1404, 1400)

In [105]:
def plyr_pass(soup):
    year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
    table = soup.find('table', {'id':'passing'})
    rows = table.find_all('tr')
    passing_list = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            tmp_dict = {}
            for cell in cells:
                if cell['data-stat'] == 'player':
                    name = cell.find('a').string
                    tmp_dict[cell['data-stat']] = name
                    tmp_dict['first_name'] = name.split()[0]
                    tmp_dict['last_name'] = name.split()[1]
                    tmp_dict['player_id'] = cell['data-append-csv'].lower()
                elif cell['data-stat'] == 'team':
                    try:
                        tmp_dict[cell['data-stat']] = cell.find('a')['title']
                        tmp_dict['team_id'] = cell.find('a').string.lower()
                    except:
                        tmp_dict[cell['data-stat']] = 'other'
                        tmp_dict['team_id'] = 'other'
                elif cell['data-stat'] == 'pos':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 'no_pos'
                    else:
                        tmp_dict[cell['data-stat']] = val
                elif cell['data-stat'] == 'qb_rec':
                    if cell.string is None:
                        tmp_dict[cell['data-stat']] = 0
                    else:
                        tmp_dict[cell['data-stat']] = float(cell['csk'])
                elif cell['data-stat']!='qbr':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    elif val.isnumeric():
                        tmp_dict[cell['data-stat']] = int(val)
                    else:
                        try:
                            tmp_dict[cell['data-stat']] = float(val)
                        except:
                            tmp_dict[cell['data-stat']] = val
                tmp_dict['year'] = year
            passing_list.append(tmp_dict)
    return passing_list

### Kick and Punt Returns

In [134]:
for yr in range(2015,1969,-1):
    pg_url = os.path.join(bs_pg,str(yr),bs_returns)
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    returns_list += plyr_ret(soup)
    time.sleep(1)

In [135]:
df_ret = pd.DataFrame.from_dict(returns_list)
df_ret.rename(columns=rename_dict, inplace=True)
df_ret.to_feather(f'{PATH}df_players_scrape_returns.feather')
df_ret.shape, df_ret.player_id.nunique(), df_ret.player.nunique()

((11382, 22), 5128, 5043)

In [130]:
def plyr_ret(soup):
    year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
    table = soup.find('table', {'id':'returns'})
    rows = table.find_all('tr')
    returns_list = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            tmp_dict = {}
            for cell in cells:
                if cell['data-stat'] == 'player':
                    name = cell.find('a').string
                    tmp_dict[cell['data-stat']] = name
                    tmp_dict['first_name'] = name.split()[0]
                    tmp_dict['last_name'] = name.split()[1]
                    tmp_dict['player_id'] = cell['data-append-csv'].lower()
                elif cell['data-stat'] == 'team':
                    try:
                        tmp_dict[cell['data-stat']] = cell.find('a')['title']
                        tmp_dict['team_id'] = cell.find('a').string.lower()
                    except:
                        tmp_dict[cell['data-stat']] = 'other'
                        tmp_dict['team_id'] = 'other'
                elif cell['data-stat'] == 'pos':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 'no_pos'
                    else:
                        tmp_dict[cell['data-stat']] = val
                else:
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    elif val.isnumeric():
                        tmp_dict[cell['data-stat']] = int(val)
                    else:
                        try:
                            tmp_dict[cell['data-stat']] = float(val)
                        except:
                            tmp_dict[cell['data-stat']] = val
                tmp_dict['year'] = year
            returns_list.append(tmp_dict)
    return returns_list

### Defense

In [158]:
for yr in range(2015,1969,-1):
    pg_url = os.path.join(bs_pg,str(yr),bs_defense)
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    defense_list += plyr_def(soup)
    time.sleep(1)

In [160]:
df_def = pd.DataFrame.from_dict(defense_list)
df_def.rename(columns=rename_dict, inplace=True)
df_def.to_feather(f'{PATH}df_players_scrape_defense.feather')
df_def.shape, df_def.player_id.nunique(), df_def.player.nunique()

((46359, 25), 12341, 11936)

In [155]:
def plyr_def(soup):
    year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
    table = soup.find('table', {'id':'defense'})
    rows = table.find_all('tr')
    defense_list = []
    missing_set = set(['sacks','tackles_solo','tackles_assists','safety_md'])
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            tmp_dict = {}
            for cell in cells:
                if cell['data-stat'] == 'player':
                    name = cell.find('a').string
                    tmp_dict[cell['data-stat']] = name
                    tmp_dict['first_name'] = name.split()[0]
                    tmp_dict['last_name'] = name.split()[1]
                    tmp_dict['player_id'] = cell['data-append-csv'].lower()
                elif cell['data-stat'] == 'team':
                    try:
                        tmp_dict[cell['data-stat']] = cell.find('a')['title']
                        tmp_dict['team_id'] = cell.find('a').string.lower()
                    except:
                        tmp_dict[cell['data-stat']] = 'other'
                        tmp_dict['team_id'] = 'other'
                elif cell['data-stat'] == 'pos':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 'no_pos'
                    else:
                        tmp_dict[cell['data-stat']] = val
                else:
                    val = cell.string
                    if val is None: 
                        if cell['data-stat'] in missing_set and year<= 1980:
                            tmp_dict[cell['data-stat']] = -999
                        else:
                            tmp_dict[cell['data-stat']] = 0
                    elif val.isnumeric():
                        tmp_dict[cell['data-stat']] = int(val)
                    else:
                        try:
                            tmp_dict[cell['data-stat']] = float(val)
                        except:
                            tmp_dict[cell['data-stat']] = val
                tmp_dict['year'] = year
            defense_list.append(tmp_dict)
    return defense_list

### Rushing

In [163]:
year = 2017
pg_url = os.path.join(bs_pg,str(year),bs_rushing)
pg = urllib.request.urlopen(pg_url)
soup = BeautifulSoup(pg, 'html.parser')

In [165]:
year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
table = soup.find('table', {'id':'rushing_and_receiving'})
rows = table.find_all('tr')
rushing_list = []
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        break

In [220]:
for yr in range(1960,1959,-1): #1960
    pg_url = os.path.join(bs_pg,str(yr),bs_rushing)
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    rushing_list += plyr_rush(soup)
    time.sleep(1)

In [221]:
df_rush = pd.DataFrame.from_dict(rushing_list)
df_rush.rename(columns=rename_dict, inplace=True)
df_rush.to_feather(f'{PATH}df_players_scrape_rushing.feather')
df_rush.shape, df_rush.player_id.nunique(), df_rush.player.nunique(), df_rush.year.min(), df_rush.year.max()

((15957, 33), 4347, 4289, 1960, 2017)

In [204]:
def plyr_rush(soup):
    year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
    table = soup.find('table', {'id':'rushing_and_receiving'})
    rows = table.find_all('tr')
    rushing_list = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            tmp_dict = {}
            for cell in cells:
                if cell['data-stat'] == 'player':
                    name = cell.find('a').string
                    tmp_dict[cell['data-stat']] = name
                    tmp_dict['first_name'] = name.split()[0]
                    tmp_dict['last_name'] = name.split()[1]
                    tmp_dict['player_id'] = cell['data-append-csv'].lower()
                elif cell['data-stat'] == 'team':
                    try:
                        tmp_dict[cell['data-stat']] = cell.find('a')['title']
                        tmp_dict['team_id'] = cell.find('a').string.lower()
                    except:
                        tmp_dict[cell['data-stat']] = 'other'
                        tmp_dict['team_id'] = 'other'
                elif cell['data-stat'] == 'pos':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 'no_pos'
                    else:
                        tmp_dict[cell['data-stat']] = val
                elif cell['data-stat'] =='catch_pct':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    else:
                        tmp_dict[cell['data-stat']] = float(val[:-1])
                else:
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    elif val.isnumeric():
                        tmp_dict[cell['data-stat']] = int(val)
                    else:
                        try:
                            tmp_dict[cell['data-stat']] = float(val)
                        except:
                            tmp_dict[cell['data-stat']] = val
                if 'targets' not in tmp_dict:
                    tmp_dict['targets'] = -999
                tmp_dict['year'] = year
            rushing_list.append(tmp_dict)
    return rushing_list

### Kicking

In [222]:
year = 2017
pg_url = os.path.join(bs_pg,str(year),bs_kicking)
pg = urllib.request.urlopen(pg_url)
soup = BeautifulSoup(pg, 'html.parser')

In [223]:
year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
table = soup.find('table', {'id':'kicking'})
rows = table.find_all('tr')
rushing_list = []
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        break

In [234]:
for yr in range(2014,1959,-1): #1960
    pg_url = os.path.join(bs_pg,str(yr),bs_kicking)
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    kicking_list += plyr_kick(soup)
    time.sleep(1)

In [239]:
df_kick = pd.DataFrame.from_dict(kicking_list)
df_kick.rename(columns=rename_dict, inplace=True)
df_kick.to_feather(f'{PATH}df_players_scrape_kicking.feather')
df_kick.shape, df_kick.player_id.nunique(), df_kick.player.nunique(), df_kick.year.min(), df_kick.year.max()

((3802, 32), 785, 785, 1960, 2017)

In [225]:
def plyr_kick(soup):
    year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
    table = soup.find('table', {'id':'kicking'})
    rows = table.find_all('tr')
    kicking_list = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            tmp_dict = {}
            for cell in cells:
                if cell['data-stat'] == 'player':
                    name = cell.find('a').string
                    tmp_dict[cell['data-stat']] = name
                    tmp_dict['first_name'] = name.split()[0]
                    tmp_dict['last_name'] = name.split()[1]
                    tmp_dict['player_id'] = cell['data-append-csv'].lower()
                elif cell['data-stat'] == 'team':
                    try:
                        tmp_dict[cell['data-stat']] = cell.find('a')['title']
                        tmp_dict['team_id'] = cell.find('a').string.lower()
                    except:
                        tmp_dict[cell['data-stat']] = 'other'
                        tmp_dict['team_id'] = 'other'
                elif cell['data-stat'] == 'pos':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 'no_pos'
                    else:
                        tmp_dict[cell['data-stat']] = val
                elif cell['data-stat'] in ['fg_perc','xp_perc']:
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    else:
                        tmp_dict[cell['data-stat']] = float(val[:-1])
                else:
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    elif val.isnumeric():
                        tmp_dict[cell['data-stat']] = int(val)
                    else:
                        try:
                            tmp_dict[cell['data-stat']] = float(val)
                        except:
                            tmp_dict[cell['data-stat']] = val
                tmp_dict['year'] = year
            kicking_list.append(tmp_dict)
    return kicking_list

### Receiving

In [19]:
yr = 2017
# pg_url = os.path.join(bs_pg,str(year),bs_receiving)
pg_url = '/'.join([bs_pg,str(yr),bs_receiving])
pg = urllib.request.urlopen(pg_url)
soup = BeautifulSoup(pg, 'html.parser')

In [21]:
year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
table = soup.find('table', {'id':'receiving'})
rows = table.find_all('tr')
rushing_list = []
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        break
cells

[<td class="left " csk="Landry,Jarvis" data-append-csv="LandJa00" data-stat="player"><a href="/players/L/LandJa00.htm">Jarvis Landry</a>*</td>,
 <td class="left " data-stat="team"><a href="/teams/mia/2017.htm" title="Miami Dolphins">MIA</a></td>,
 <td class="right " data-stat="age">25</td>,
 <td class="left " data-stat="pos">WR</td>,
 <td class="right " data-stat="g">16</td>,
 <td class="right " data-stat="gs">16</td>,
 <td class="right " data-stat="targets">161</td>,
 <td class="right " data-stat="rec">112</td>,
 <td class="right " data-stat="catch_pct">69.6%</td>,
 <td class="right " data-stat="rec_yds">987</td>,
 <td class="right " data-stat="rec_yds_per_rec">8.8</td>,
 <td class="right " data-stat="rec_td">9</td>,
 <td class="right " data-stat="rec_long">49</td>,
 <td class="right " data-stat="rec_per_g">7.0</td>,
 <td class="right " data-stat="rec_yds_per_g">61.7</td>,
 <td class="right " data-stat="fumbles">4</td>]

In [36]:
for yr in range(2015,1959,-1): #1960
#     pg_url = os.path.join(bs_pg,str(yr),bs_receiving)
    pg_url = '/'.join([bs_pg,str(yr),bs_receiving])
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    receiving_list += plyr_recv(soup)
    time.sleep(1)

In [37]:
df_recv = pd.DataFrame.from_dict(receiving_list)
df_recv.rename(columns=rename_dict, inplace=True)
df_recv.to_feather(f'{PATH}data/interim/df_players_scrape_receiving.feather')
df_recv.shape, df_recv.player_id.nunique(), df_recv.player.nunique(), df_recv.year.min(), df_recv.year.max()

((20109, 21), 5181, 5094, 1960, 2017)

In [23]:
def plyr_recv(soup):
    year = int(soup.find('h1',{'itemprop':'name'}).find('span').string)
    table = soup.find('table', {'id':'receiving'})
    rows = table.find_all('tr')
    receiving_list = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            tmp_dict = {}
            for cell in cells:
                if cell['data-stat'] == 'player':
                    name = cell.find('a').string
                    tmp_dict[cell['data-stat']] = name
                    tmp_dict['first_name'] = name.split()[0]
                    tmp_dict['last_name'] = name.split()[1]
                    tmp_dict['player_id'] = cell['data-append-csv'].lower()
                elif cell['data-stat'] == 'team':
                    try:
                        tmp_dict[cell['data-stat']] = cell.find('a')['title']
                        tmp_dict['team_id'] = cell.find('a').string.lower()
                    except:
                        tmp_dict[cell['data-stat']] = 'other'
                        tmp_dict['team_id'] = 'other'
                elif cell['data-stat'] == 'pos':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 'no_pos'
                    else:
                        tmp_dict[cell['data-stat']] = val
                elif cell['data-stat'] =='catch_pct':
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    else:
                        tmp_dict[cell['data-stat']] = float(val[:-1])
                else:
                    val = cell.string
                    if val is None: 
                        tmp_dict[cell['data-stat']] = 0
                    elif val.isnumeric():
                        tmp_dict[cell['data-stat']] = int(val)
                    else:
                        try:
                            tmp_dict[cell['data-stat']] = float(val)
                        except:
                            tmp_dict[cell['data-stat']] = val
                if 'catch_pct' not in tmp_dict:
                    tmp_dict['catch_pct'] = -999
                tmp_dict['year'] = year
            receiving_list.append(tmp_dict)
    return receiving_list

## Merge the data

In [55]:
df_rec = pd.read_feather(f'{PATH}df_players_scrape_receiving.feather')
df_rush = pd.read_feather(f'{PATH}df_players_scrape_rushing.feather')
df_kick = pd.read_feather(f'{PATH}df_players_scrape_kicking.feather')
df_pas = pd.read_feather(f'{PATH}df_players_scrape_passing.feather')
df_ret = pd.read_feather(f'{PATH}df_players_scrape_returns.feather')
df_scr = pd.read_feather(f'{PATH}df_players_scrape_scoring.feather')
df_def = pd.read_feather(f'{PATH}df_players_scrape_defense.feather')

In [319]:
# df_rec.fillna(value = -999, inplace=True)
# df_rush.fillna(value = -999, inplace=True)
# df_kick.fillna(value = -999, inplace=True)
# df_pas.fillna(value = -999, inplace=True)
# df_ret.fillna(value = -999, inplace=True)
# df_scr.fillna(value = -999, inplace=True)
# df_def.fillna(value = -999, inplace=True)
# df_rec['id'] = df_rec.player_id + '_' + df_rec.year.astype(str)
# df_rush['id'] = df_rush.player_id + '_' + df_rush.year.astype(str)
# df_kick['id'] = df_kick.player_id + '_' + df_kick.year.astype(str)
# df_pas['id'] = df_pas.player_id + '_' + df_pas.year.astype(str)
# df_ret['id'] = df_ret.player_id + '_' + df_ret.year.astype(str)
# df_scr['id'] = df_scr.player_id + '_' + df_scr.year.astype(str)
# df_def['id'] = df_def.player_id + '_' + df_def.year.astype(str)
df_plyr_merge = df_rec.merge(df_rush, how='outer')
df_plyr_merge = df_plyr_merge.merge(df_kick, how='outer')
df_plyr_merge = df_plyr_merge.merge(df_pas, how='outer')
df_ret_tmp = df_ret.drop(columns='all_purpose_yds')
df_plyr_merge = df_plyr_merge.merge(df_ret_tmp, how='outer')
df_plyr_merge.sort_values('id', inplace=True)
df_ret.sort_values('id', inplace=True)
na_list = df_plyr_merge.loc[(df_plyr_merge.id.isin(df_ret.id.tolist())) & (df_plyr_merge.all_purpose_yds.isna()),'id'].tolist()
na_values = df_ret.loc[df_ret.id.isin(na_list),'all_purpose_yds'].tolist()
df_plyr_merge.loc[(df_plyr_merge.id.isin(df_ret.id.tolist())) & (df_plyr_merge.all_purpose_yds.isna()),'all_purpose_yds'] = na_values

df_scr_tmp = df_scr.drop(columns=['rush_td','rec_td','fga','fgm','xpa','xpm'])
df_plyr_merge = df_plyr_merge.merge(df_scr_tmp, how='outer')
df_plyr_merge = df_plyr_merge.sort_values('id').reset_index(drop=True)
df_scr = df_scr.sort_values('id').reset_index(drop=True)
comp_df = df_scr
for vnm in ['rush_td','rec_td','fga','fgm','xpa','xpm']:
    na_list = df_plyr_merge.loc[(df_plyr_merge.id.isin(comp_df.id.tolist())) & (df_plyr_merge[vnm].isna()),'id'].tolist()
    na_values = comp_df.loc[comp_df.id.isin(na_list),vnm].tolist()
    df_plyr_merge.loc[(df_plyr_merge.id.isin(comp_df.id.tolist())) & (df_plyr_merge[vnm].isna()),vnm] = na_values

    
def_var_list = ['fumbles','safety_md']    
df_def_tmp = df_def.drop(columns=def_var_list)

df_plyr_merge = df_plyr_merge.merge(df_def_tmp, how='outer')
df_plyr_merge = df_plyr_merge.sort_values('id').reset_index(drop=True)
df_def = df_def.sort_values('id').reset_index(drop=True)
comp_df = df_def
for vnm in def_var_list:
    na_list = df_plyr_merge.loc[(df_plyr_merge.id.isin(comp_df.id.tolist())) & (df_plyr_merge[vnm].isna()),'id'].tolist()
    na_values = comp_df.loc[comp_df.id.isin(na_list),vnm].tolist()
    df_plyr_merge.loc[(df_plyr_merge.id.isin(comp_df.id.tolist())) & (df_plyr_merge[vnm].isna()),vnm] = na_values

In [320]:
df_dup = df_plyr_merge.loc[df_plyr_merge.duplicated(subset=['id'], keep=False),:].sort_values(['player_id','year'])
df_dup.shape

(0, 110)

In [321]:
# df_plyr_merge.loc[(df_plyr_merge.id=='abbrja00_2017'),int_list]
# df_def.loc[(df_def.id=='abbrja00_2017'),int_list]
# int_list = list(set(df_plyr_merge.columns).intersection(set(df_def.columns)))

In [86]:
df_plyr_merge = df_recv.merge(df_rush, how='outer')
df_plyr_merge = df_plyr_merge.merge(df_kick, how='outer')
df_plyr_merge = df_plyr_merge.merge(df_pas, how='outer')
df_plyr_merge = df_plyr_merge.merge(df_ret, how='outer')
df_plyr_merge = df_plyr_merge.merge(df_scr, how='outer')
df_plyr_merge = df_plyr_merge.merge(df_def, how='outer')
df_plyr_merge.fillna(-888, inplace=True)
df_plyr_merge.shape

(110338, 109)

In [87]:
df_plyr_merge.head()

,age,catch_pct,first_name,fumbles,games,games_started,last_name,player,player_id,pos,rec,rec_long,rec_per_g,rec_td,rec_yds,rec_yds_per_g,rec_yds_per_rec,targets,team,team_id,year,all_purpose_yds,all_purpose_yds_per_touch,rush_att,rush_att_per_g,rush_long,rush_receive_td,rush_td,rush_yds,rush_yds_per_att,rush_yds_per_g,touches,yds_from_scrimmage,fg_perc,fga,fga1,fga2,fga3,fga4,fga5,fgm,fgm1,fgm2,fgm3,fgm4,fgm5,punt,punt_blocked,punt_long,punt_yds,...,pass_cmp,pass_cmp_perc,pass_int,pass_int_perc,pass_long,pass_net_yds_per_att,pass_rating,pass_sacked,pass_sacked_perc,pass_sacked_yds,pass_td,pass_td_perc,pass_yds,pass_yds_per_att,pass_yds_per_cmp,pass_yds_per_g,qb_rec,kick_ret,kick_ret_long,kick_ret_td,kick_ret_yds,kick_ret_yds_per_ret,punt_ret,punt_ret_long,punt_ret_td,punt_ret_yds,punt_ret_yds_per_ret,all_td,int_td,fum_td,ko_td,block_rtd,points_per_g,punt_td,safety_md,scoring,two_pt_att,two_pt_md,def_int,def_int_long,def_int_td,def_int_yds,fumbles_forced,fumbles_rec,fumbles_rec_td,fumbles_rec_yds,pass_defended,sacks,tackles_assists,tackles_solo
0,25,69.6,Jarvis,4.0,16,16,Landry,Jarvis Landry,landja00,WR,112.0,49.0,7.0,9.0,987.0,61.7,8.8,161.0,Miami Dolphins,mia,2017,1056.0,8.4,1.0,0.1,-7.0,9.0,0.0,-7.0,-7.0,-0.4,125.0,980.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,0.0,0.0,0.0,0.0,0.0,0.0,39.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,-5.0,0.0,12.0,16.0,0.0,81.0,6.8,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0
1,34,67.7,Larry,1.0,16,16,Fitzgerald,Larry Fitzgerald,fitzla00,WR,109.0,37.0,6.8,6.0,1156.0,72.3,10.6,161.0,Arizona Cardinals,ari,2017,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,1.0,100.0,0.0,0.0,21.0,21.0,118.7,0.0,0.0,0.0,0.0,0.0,21.0,21.0,21.0,1.3,0.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0
2,24,69.8,Michael,0.0,16,14,Thomas,Michael Thomas,thommi05,WR,104.0,43.0,6.5,5.0,1245.0,77.8,12.0,149.0,New Orleans Saints,nor,2017,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0
3,25,64.2,Keenan,1.0,16,15,Allen,Keenan Allen,alleke00,WR,102.0,51.0,6.4,6.0,1393.0,87.1,13.7,159.0,Los Angeles Chargers,lac,2017,1461.0,14.0,2.0,0.1,6.0,6.0,0.0,9.0,4.5,0.6,104.0,1402.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0
4,29,62.0,Antonio,4.0,14,14,Brown,Antonio Brown,browan04,WR,101.0,57.0,7.2,9.0,1533.0,109.5,15.2,163.0,Pittsburgh Steelers,pit,2017,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,-888.0,-888.0,-888.0,

In [322]:
df_plyr_merge.to_feather(f'{PATH}df_plyr_merge.feather')

### Add scoring

In [89]:
df_ff_score = df_plyr_merge.copy()
for i,v in df_ff_score.dtypes.iteritems():
    if v in ('float64','int64'):
        df_ff_score.loc[df_ff_score[i]<=-888,i] = 0

In [95]:
print(list(df_ff_score.columns))

['age', 'catch_pct', 'first_name', 'fumbles', 'games', 'games_started', 'last_name', 'player', 'player_id', 'pos', 'rec', 'rec_long', 'rec_per_g', 'rec_td', 'rec_yds', 'rec_yds_per_g', 'rec_yds_per_rec', 'targets', 'team', 'team_id', 'year', 'all_purpose_yds', 'all_purpose_yds_per_touch', 'rush_att', 'rush_att_per_g', 'rush_long', 'rush_receive_td', 'rush_td', 'rush_yds', 'rush_yds_per_att', 'rush_yds_per_g', 'touches', 'yds_from_scrimmage', 'fg_perc', 'fga', 'fga1', 'fga2', 'fga3', 'fga4', 'fga5', 'fgm', 'fgm1', 'fgm2', 'fgm3', 'fgm4', 'fgm5', 'punt', 'punt_blocked', 'punt_long', 'punt_yds', 'punt_yds_per_punt', 'xp_perc', 'xpa', 'xpm', 'comebacks', 'game_winning_drives', 'pass_adj_net_yds_per_att', 'pass_adj_yds_per_att', 'pass_att', 'pass_cmp', 'pass_cmp_perc', 'pass_int', 'pass_int_perc', 'pass_long', 'pass_net_yds_per_att', 'pass_rating', 'pass_sacked', 'pass_sacked_perc', 'pass_sacked_yds', 'pass_td', 'pass_td_perc', 'pass_yds', 'pass_yds_per_att', 'pass_yds_per_cmp', 'pass_yds_p

In [97]:
passing_score = df_ff_score.pass_yds*0.04 + df_ff_score.pass_td*4 + df_ff_score.pass_int*-2 + df_ff_score.two_pt_md*2
rushing_score = df_ff_score.rush_yds*0.1 + df_ff_score.rush_td*6 
rec_score = df_ff_score.rec_yds*0.1 + df_ff_score.rec_td*6
misc_score = df_ff_score.kick_ret_td*6 + df_ff_score.punt_ret_td*6 + df_ff_score.fumbles_rec_td*6 + df_ff_score.fumbles*-2           
kicking_score  = (df_ff_score.xpm*2 + df_ff_score.xpa*-1 + df_ff_score.fga*-1 + df_ff_score.fgm*4
                  + df_ff_score.fgm4 + df_ff_score.fgm5*2)
df_plyr_merge['ff_score'] = passing_score + rushing_score + rec_score + misc_score + kicking_score

In [98]:
df_plyr_merge.columns

Index(['age', 'catch_pct', 'first_name', 'fumbles', 'games', 'games_started',
       'last_name', 'player', 'player_id', 'pos',
       ...
       'def_int_yds', 'fumbles_forced', 'fumbles_rec', 'fumbles_rec_td',
       'fumbles_rec_yds', 'pass_defended', 'sacks', 'tackles_assists',
       'tackles_solo', 'ff_score'],
      dtype='object', length=110)

In [100]:
df_plyr_merge.to_feather(f'{PATH}data/interim/df_plyr_merge_score.feather')

In [102]:
df_plyr_merge.loc[(df_plyr_merge['ff_score']>0) & (df_plyr_merge['games']==1) ,:]

,age,catch_pct,first_name,fumbles,games,games_started,last_name,player,player_id,pos,rec,rec_long,rec_per_g,rec_td,rec_yds,rec_yds_per_g,rec_yds_per_rec,targets,team,team_id,year,all_purpose_yds,all_purpose_yds_per_touch,rush_att,rush_att_per_g,rush_long,rush_receive_td,rush_td,rush_yds,rush_yds_per_att,rush_yds_per_g,touches,yds_from_scrimmage,fg_perc,fga,fga1,fga2,fga3,fga4,fga5,fgm,fgm1,fgm2,fgm3,fgm4,fgm5,punt,punt_blocked,punt_long,punt_yds,...,pass_cmp_perc,pass_int,pass_int_perc,pass_long,pass_net_yds_per_att,pass_rating,pass_sacked,pass_sacked_perc,pass_sacked_yds,pass_td,pass_td_perc,pass_yds,pass_yds_per_att,pass_yds_per_cmp,pass_yds_per_g,qb_rec,kick_ret,kick_ret_long,kick_ret_td,kick_ret_yds,kick_ret_yds_per_ret,punt_ret,punt_ret_long,punt_ret_td,punt_ret_yds,punt_ret_yds_per_ret,all_td,int_td,fum_td,ko_td,block_rtd,points_per_g,punt_td,safety_md,scoring,two_pt_att,two_pt_md,def_int,def_int_long,def_int_td,def_int_yds,fumbles_forced,fumbles_rec,fumbles_rec_td,fumbles_rec_yds,pass_defended,sacks,tackles_assists,tackles_solo,ff_score
322,26,66.7,David,2.0,1,1,Johnson,David Johnson,johnda08,rb,6.0,24.0,6.0,0.0,67.0,67.0,11.2,9.0,Arizona Cardinals,ari,2017,90.0,5.3,11.0,11.0,6.0,0.0,0.0,23.0,2.1,23.0,17.0,90.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,5.0
392,27,75.0,Nick,0.0,1,0,Williams,Nick Williams,willni02,no_pos,3.0,14.0,3.0,0.0,30.0,30.0,10.0,4.0,Atlanta Falcons,atl,2017,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,3.0
396,23,100.0,Emanuel,0.0,1,0,Byrd,Emanuel Byrd,byrdem00,no_pos,2.0,29.0,2.0,0.0,31.0,31.0,15.5,2.0,Green Bay Packers,gnb,2017,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,3.1
413,26,100.0,Marcus,0.0,1,0,Murphy,Marcus Murphy,murpma03,no_pos,2.0,5.0,2.0,0.0,7.0,7.0,3.5,2.0,Buffalo Bills,buf,2017,66.0,6.6,7.0,7.0,25.0,0.0,0.0,41.0,5.9,41.0,10.0,48.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,...,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,1.0,18.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0,0.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,4.8
416,23,50.0,Darius,0.0,1,1,Powe,Darius Powe,poweda02,wr,2.0,8.0,2.0,0.0,13.0,13.0,6.5,4.0,New York Giants,nyg,2017,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.0,-888.

In [106]:
df_plyr_merge.loc[df_plyr_merge.player=='Tom Brady',['year','age','ff_score','player_id']].sort_values('year')

,year,age,ff_score,player_id
33631,2000,23,0.24,bradto00
86678,2001,24,-24.00,bradto00
7454,2001,24,144.32,bradto00
85552,2002,25,-22.00,bradto00
26866,2002,25,229.56,bradto00
47454,2002,25,6.00,bradto00
26762,2003,26,199.10,bradto00
82667,2004,27,-14.00,bradto00
26665,2004,27,220.48,bradto00
26583,2005,28,247.30,bradto00


# Extra